# G-Retrieval Style Comparison: LPG (GAT) vs RDF (TransE / DistMult)

This notebook trains graph encoders on the FinDER dual-graph PyG dataset and evaluates
how well learned graph embeddings capture answer-relevant information.

**Models:**
- **GAT** (Graph Attention Network) on LPG subgraphs — 384d node features + message passing
- **TransE** on RDF triples — translation-based: h + r ≈ t (asymmetric)
- **DistMult** on RDF triples — bilinear: h · diag(r) · t (symmetric)

**Evaluation:**
- Link prediction (self-supervised training objective)
- Graph→Answer retrieval (cosine similarity with sentence embeddings)
- Category-wise and graph-size breakdown

In [ ]:
# Cell 1: Setup & Install
# Uncomment for Colab:
# !pip install torch torch-geometric sentence-transformers rouge-score
# !pip install torch_scatter torch_sparse -f https://data.pyg.org/whl/torch-2.1.0+cu121.html

import sys, os, json, warnings
from pathlib import Path
from collections import defaultdict
from dataclasses import dataclass, field
from typing import Dict, List, Optional, Tuple

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib
matplotlib.rcParams.update({'font.size': 11, 'figure.dpi': 120})
warnings.filterwarnings('ignore', category=FutureWarning)

import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import DataLoader

from torch_geometric.nn import GATConv, global_mean_pool
from torch_geometric.nn.kge import TransE, DistMult
from torch_geometric.utils import negative_sampling, scatter

# Project imports — adjust path for Colab
PROJECT_ROOT = Path('..').resolve()
if str(PROJECT_ROOT) not in sys.path:
    sys.path.insert(0, str(PROJECT_ROOT))

from src.data import FinDERGraphQADataset, DualGraphBatch, dual_graph_collate_fn, VocabularyBuilder

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f'Device: {device}')
if device.type == 'cuda':
    print(f'  GPU: {torch.cuda.get_device_name()}')
    print(f'  Memory: {torch.cuda.get_device_properties(0).total_memory / 1e9:.1f} GB')

In [ ]:
# Cell 2: Data Loading
DATA_ROOT = PROJECT_ROOT / 'data' / 'processed' / 'finder_pyg'

train_ds = FinDERGraphQADataset(root=str(DATA_ROOT), split='train')
val_ds   = FinDERGraphQADataset(root=str(DATA_ROOT), split='val')
test_ds  = FinDERGraphQADataset(root=str(DATA_ROOT), split='test')

BATCH_SIZE = 32
train_loader = DataLoader(train_ds, batch_size=BATCH_SIZE, shuffle=True,
                          collate_fn=dual_graph_collate_fn, num_workers=0, pin_memory=True)
val_loader   = DataLoader(val_ds, batch_size=BATCH_SIZE, shuffle=False,
                          collate_fn=dual_graph_collate_fn, num_workers=0)
test_loader  = DataLoader(test_ds, batch_size=BATCH_SIZE, shuffle=False,
                          collate_fn=dual_graph_collate_fn, num_workers=0)

# Load vocabularies
vocabs = FinDERGraphQADataset.get_vocab(root=str(DATA_ROOT))
metadata = json.loads((DATA_ROOT / 'processed' / 'metadata.json').read_text())

NUM_RDF_ENTITIES  = metadata['vocab_sizes']['rdf_entities']   # 17,534
NUM_RDF_RELATIONS = metadata['vocab_sizes']['rdf_relations']  # 4,340
LPG_FEATURE_DIM   = metadata['lpg_feature_dim']               # 384

# Dataset statistics
print(f"\n{'='*50}")
print(f"Dataset Statistics")
print(f"{'='*50}")
print(f"  Train: {len(train_ds):,} | Val: {len(val_ds):,} | Test: {len(test_ds):,}")
print(f"  LPG feature dim: {LPG_FEATURE_DIM}")
print(f"  RDF entities: {NUM_RDF_ENTITIES:,} | relations: {NUM_RDF_RELATIONS:,}")

# Category distribution
categories = [train_ds[i].category for i in range(len(train_ds))]
cat_counts = pd.Series(categories).value_counts()
print(f"\nCategory distribution (train):")
for cat, count in cat_counts.items():
    print(f"  {cat}: {count}")

# Average graph sizes
lpg_nodes = [train_ds[i].lpg_num_nodes.item() for i in range(min(200, len(train_ds)))]
rdf_edges = [train_ds[i].rdf_edge_index.shape[1] for i in range(min(200, len(train_ds)))]
print(f"\nAvg LPG nodes/sample: {np.mean(lpg_nodes):.1f} (±{np.std(lpg_nodes):.1f})")
print(f"Avg RDF triples/sample: {np.mean(rdf_edges):.1f} (±{np.std(rdf_edges):.1f})")

In [ ]:
# Cell 3: Model Definitions

# --- GAT for LPG (batched) ---

class BatchedGAT(nn.Module):
    """GAT encoder for LPG subgraphs with batched graph-level pooling.

    Based on MessagePassingGNN from src/_legacy/models.py but uses
    global_mean_pool for proper mini-batch support.
    """

    def __init__(
        self,
        input_dim: int = 384,
        hidden_dim: int = 256,
        output_dim: int = 384,
        num_layers: int = 2,
        heads: int = 4,
        dropout: float = 0.1,
    ):
        super().__init__()
        self.input_proj = nn.Linear(input_dim, hidden_dim)
        self.convs = nn.ModuleList()
        self.norms = nn.ModuleList()
        for i in range(num_layers):
            in_ch = hidden_dim if i == 0 else hidden_dim * heads
            self.convs.append(GATConv(in_ch, hidden_dim, heads=heads, dropout=dropout))
            self.norms.append(nn.LayerNorm(hidden_dim * heads))
        self.output_proj = nn.Linear(hidden_dim * heads, output_dim)
        self.dropout = nn.Dropout(dropout)

    def forward(self, x, edge_index, batch):
        """Forward pass with batched graph-level pooling.

        Args:
            x: [sum(N_i), input_dim] node features
            edge_index: [2, sum(E_i)] COO edges
            batch: [sum(N_i)] graph membership index

        Returns:
            [B, output_dim] graph-level embeddings
        """
        x = torch.relu(self.input_proj(x))
        for conv, norm in zip(self.convs, self.norms):
            x = conv(x, edge_index)
            x = norm(x)
            x = torch.relu(x)
            x = self.dropout(x)
        node_emb = self.output_proj(x)  # [sum(N_i), output_dim]
        return global_mean_pool(node_emb, batch)  # [B, output_dim]

    def get_node_embeddings(self, x, edge_index):
        """Get per-node embeddings (no pooling). For link prediction decoding."""
        x = torch.relu(self.input_proj(x))
        for conv, norm in zip(self.convs, self.norms):
            x = conv(x, edge_index)
            x = norm(x)
            x = torch.relu(x)
            x = self.dropout(x)
        return self.output_proj(x)  # [sum(N_i), output_dim]


# --- KGE for RDF (TransE / DistMult) with per-graph aggregation ---

class BatchedKGE(nn.Module):
    """KGE encoder for RDF triples with per-graph embedding aggregation.

    Learns global entity and relation embeddings, then for each question's
    RDF subgraph, aggregates triple representations via scatter mean.
    """

    def __init__(
        self,
        model_type: str,  # 'transe' or 'distmult'
        num_entities: int,
        num_relations: int,
        hidden_dim: int = 256,
        output_dim: int = 384,
    ):
        super().__init__()
        self.model_type = model_type
        self.hidden_dim = hidden_dim
        self.output_dim = output_dim

        if model_type == 'transe':
            self.kge = TransE(
                num_nodes=num_entities,
                num_relations=num_relations,
                hidden_channels=hidden_dim,
                margin=1.0,
                p_norm=1.0,
            )
        elif model_type == 'distmult':
            self.kge = DistMult(
                num_nodes=num_entities,
                num_relations=num_relations,
                hidden_channels=hidden_dim,
            )
        else:
            raise ValueError(f"Unknown model_type: {model_type}")

        self.output_proj = nn.Linear(hidden_dim, output_dim)

    def loss(self, head_index, rel_type, tail_index):
        """KGE training loss with built-in negative sampling."""
        return self.kge.loss(head_index, rel_type, tail_index)

    def forward(self, batch: 'DualGraphBatch'):
        """Compute per-graph embeddings from RDF triples.

        For each triple (h, r, t) in the batch, computes h_emb + r_emb,
        then aggregates per-graph via scatter mean on the head node's
        graph membership.

        Returns:
            [B, output_dim] graph-level embeddings
        """
        edge_index = batch.rdf_edge_index  # [2, sum(T_i)]
        edge_type = batch.rdf_edge_type    # [sum(T_i)]
        rdf_batch = batch.rdf_batch        # [sum(N_rdf_i)]
        global_idx = batch.rdf_global_node_idx  # [sum(N_rdf_i)]

        if edge_index.shape[1] == 0:
            return torch.zeros(batch.batch_size, self.output_dim, device=edge_index.device)

        head_local = edge_index[0]  # local node indices
        tail_local = edge_index[1]

        # Map local indices to global for embedding lookup
        head_global = global_idx[head_local]
        tail_global = global_idx[tail_local]

        head_emb = self.kge.node_emb(head_global)  # [sum(T_i), hidden_dim]
        rel_emb = self.kge.rel_emb(edge_type)       # [sum(T_i), hidden_dim]

        # Triple representation: h + r (translation-style, works for both)
        triple_emb = head_emb + rel_emb  # [sum(T_i), hidden_dim]

        # Determine graph membership for each triple (from head node)
        triple_graph = rdf_batch[head_local]  # [sum(T_i)]

        # Aggregate triples per graph (using PyG's scatter with reduce='mean')
        graph_emb = scatter(triple_emb, triple_graph, dim=0,
                            dim_size=batch.batch_size, reduce='mean')  # [B, hidden_dim]

        return self.output_proj(graph_emb)  # [B, output_dim]

    def get_entity_embeddings(self):
        """Export projected entity embeddings [num_entities, output_dim]."""
        with torch.no_grad():
            return self.output_proj(self.kge.node_emb.weight)


# Quick sanity check
print('BatchedGAT params:', sum(p.numel() for p in BatchedGAT().parameters()) / 1e3, 'K')
print('BatchedKGE (TransE) params:',
      sum(p.numel() for p in BatchedKGE('transe', NUM_RDF_ENTITIES, NUM_RDF_RELATIONS).parameters()) / 1e6, 'M')
print('BatchedKGE (DistMult) params:',
      sum(p.numel() for p in BatchedKGE('distmult', NUM_RDF_ENTITIES, NUM_RDF_RELATIONS).parameters()) / 1e6, 'M')

In [ ]:
# Cell 4: Training — GAT (LPG) via Link Prediction

def train_gat_epoch(model, loader, optimizer):
    """Train GAT via link prediction with negative sampling."""
    model.train()
    total_loss = 0
    num_batches = 0

    for batch in loader:
        batch = batch.to(device)
        optimizer.zero_grad()

        # Get node embeddings (no pooling)
        z = model.get_node_embeddings(batch.lpg_x, batch.lpg_edge_index)

        # Positive edges
        pos_edge = batch.lpg_edge_index
        num_nodes = batch.lpg_x.shape[0]

        if pos_edge.shape[1] == 0:
            continue

        # Negative sampling
        neg_edge = negative_sampling(
            pos_edge, num_nodes=num_nodes,
            num_neg_samples=pos_edge.shape[1],
        )

        # Score positive and negative edges via dot product
        pos_score = (z[pos_edge[0]] * z[pos_edge[1]]).sum(dim=-1)
        neg_score = (z[neg_edge[0]] * z[neg_edge[1]]).sum(dim=-1)

        # BCE loss
        pos_loss = F.binary_cross_entropy_with_logits(pos_score, torch.ones_like(pos_score))
        neg_loss = F.binary_cross_entropy_with_logits(neg_score, torch.zeros_like(neg_score))
        loss = pos_loss + neg_loss

        loss.backward()
        optimizer.step()

        total_loss += loss.item()
        num_batches += 1

    return total_loss / max(num_batches, 1)


@torch.no_grad()
def eval_gat_link_prediction(model, loader):
    """Evaluate GAT link prediction: MRR and Hits@10."""
    model.eval()
    mrr_sum, hits10_sum, count = 0.0, 0.0, 0

    for batch in loader:
        batch = batch.to(device)
        z = model.get_node_embeddings(batch.lpg_x, batch.lpg_edge_index)

        pos_edge = batch.lpg_edge_index
        if pos_edge.shape[1] == 0:
            continue

        num_nodes = z.shape[0]
        # Score a sample of edges (full ranking is too slow)
        sample_size = min(500, pos_edge.shape[1])
        idx = torch.randperm(pos_edge.shape[1])[:sample_size]
        src, dst = pos_edge[0, idx], pos_edge[1, idx]

        for s, d in zip(src, dst):
            # Score true tail vs all nodes
            scores = (z[s].unsqueeze(0) * z).sum(dim=-1)  # [num_nodes]
            rank = (scores >= scores[d]).sum().item()
            mrr_sum += 1.0 / rank
            hits10_sum += 1.0 if rank <= 10 else 0.0
            count += 1

    mrr = mrr_sum / max(count, 1)
    hits10 = hits10_sum / max(count, 1)
    return {'mrr': mrr, 'hits@10': hits10}


# Train GAT
GAT_EPOCHS = 50
GAT_LR = 1e-3

gat_model = BatchedGAT(input_dim=LPG_FEATURE_DIM).to(device)
gat_optimizer = torch.optim.Adam(gat_model.parameters(), lr=GAT_LR, weight_decay=1e-5)

gat_history = {'train_loss': [], 'val_mrr': [], 'val_hits10': []}
best_val_mrr = 0.0
best_gat_state = None

print(f'Training GAT for {GAT_EPOCHS} epochs...')
for epoch in range(1, GAT_EPOCHS + 1):
    loss = train_gat_epoch(gat_model, train_loader, gat_optimizer)
    gat_history['train_loss'].append(loss)

    if epoch % 5 == 0 or epoch == 1:
        val_metrics = eval_gat_link_prediction(gat_model, val_loader)
        gat_history['val_mrr'].append(val_metrics['mrr'])
        gat_history['val_hits10'].append(val_metrics['hits@10'])
        print(f'  Epoch {epoch:3d} | Loss: {loss:.4f} | Val MRR: {val_metrics["mrr"]:.4f} | Val Hits@10: {val_metrics["hits@10"]:.3f}')

        if val_metrics['mrr'] > best_val_mrr:
            best_val_mrr = val_metrics['mrr']
            best_gat_state = {k: v.cpu().clone() for k, v in gat_model.state_dict().items()}

# Restore best model
if best_gat_state:
    gat_model.load_state_dict(best_gat_state)
    gat_model.to(device)
print(f'\nBest GAT Val MRR: {best_val_mrr:.4f}')

In [ ]:
# Cell 5: Training — TransE & DistMult (RDF)

def collect_all_rdf_triples(loader):
    """Collect all (head_global, rel, tail_global) triples from the dataset."""
    heads, rels, tails = [], [], []
    for batch in loader:
        ei = batch.rdf_edge_index
        et = batch.rdf_edge_type
        gi = batch.rdf_global_node_idx
        if ei.shape[1] == 0:
            continue
        heads.append(gi[ei[0]])
        tails.append(gi[ei[1]])
        rels.append(et)
    return torch.cat(heads), torch.cat(rels), torch.cat(tails)


def train_kge_epoch(model, head, rel, tail, optimizer, batch_size=512):
    """Train KGE model for one epoch over all triples."""
    model.train()
    perm = torch.randperm(head.shape[0], device=head.device)
    total_loss = 0
    num_batches = 0

    for i in range(0, head.shape[0], batch_size):
        idx = perm[i:i+batch_size]
        optimizer.zero_grad()
        loss = model.loss(head[idx], rel[idx], tail[idx])
        loss.backward()
        optimizer.step()
        total_loss += loss.item()
        num_batches += 1

    return total_loss / max(num_batches, 1)


@torch.no_grad()
def eval_kge(model, head, rel, tail, sample_size=1000, k=10):
    """Evaluate KGE with sampled ranking: MRR and Hits@K."""
    model.eval()
    n = min(sample_size, head.shape[0])
    idx = torch.randperm(head.shape[0])[:n]
    h, r, t = head[idx], rel[idx], tail[idx]

    node_emb = model.kge.node_emb.weight  # [num_entities, hidden_dim]
    rel_emb = model.kge.rel_emb(r)        # [n, hidden_dim]
    h_emb = model.kge.node_emb(h)          # [n, hidden_dim]

    mrr_sum, hits_sum = 0.0, 0.0

    for i in range(n):
        if model.model_type == 'transe':
            # score = -||h + r - t||  (higher = better)
            pred = h_emb[i] + rel_emb[i]  # [hidden_dim]
            scores = -torch.norm(node_emb - pred.unsqueeze(0), p=1, dim=-1)  # [num_entities]
        else:  # distmult
            # score = sum(h * r * t)
            pred = h_emb[i] * rel_emb[i]  # [hidden_dim]
            scores = (node_emb * pred.unsqueeze(0)).sum(dim=-1)  # [num_entities]

        rank = (scores >= scores[t[i]]).sum().item()
        mrr_sum += 1.0 / max(rank, 1)
        hits_sum += 1.0 if rank <= k else 0.0

    return {'mrr': mrr_sum / n, f'hits@{k}': hits_sum / n}


# Collect triples
print('Collecting RDF triples...')
train_h, train_r, train_t = collect_all_rdf_triples(train_loader)
val_h, val_r, val_t = collect_all_rdf_triples(val_loader)
train_h, train_r, train_t = train_h.to(device), train_r.to(device), train_t.to(device)
val_h, val_r, val_t = val_h.to(device), val_r.to(device), val_t.to(device)
print(f'  Train triples: {train_h.shape[0]:,} | Val triples: {val_h.shape[0]:,}')

# Train both models
KGE_EPOCHS = 100
KGE_LR = 1e-2
KGE_BATCH = 512

kge_models = {}
kge_histories = {}

for model_type in ['transe', 'distmult']:
    print(f'\n{"="*50}')
    print(f'Training {model_type.upper()} for {KGE_EPOCHS} epochs...')
    print(f'{"="*50}')

    model = BatchedKGE(model_type, NUM_RDF_ENTITIES, NUM_RDF_RELATIONS).to(device)
    optimizer = torch.optim.Adam(model.parameters(), lr=KGE_LR)
    history = {'train_loss': [], 'val_mrr': [], 'val_hits10': []}
    best_mrr = 0.0
    best_state = None

    for epoch in range(1, KGE_EPOCHS + 1):
        loss = train_kge_epoch(model, train_h, train_r, train_t, optimizer, KGE_BATCH)
        history['train_loss'].append(loss)

        if epoch % 10 == 0 or epoch == 1:
            val_metrics = eval_kge(model, val_h, val_r, val_t)
            history['val_mrr'].append(val_metrics['mrr'])
            history['val_hits10'].append(val_metrics['hits@10'])
            print(f'  Epoch {epoch:3d} | Loss: {loss:.4f} | Val MRR: {val_metrics["mrr"]:.4f} | Val Hits@10: {val_metrics["hits@10"]:.3f}')

            if val_metrics['mrr'] > best_mrr:
                best_mrr = val_metrics['mrr']
                best_state = {k: v.cpu().clone() for k, v in model.state_dict().items()}

    if best_state:
        model.load_state_dict(best_state)
        model.to(device)
    print(f'Best {model_type.upper()} Val MRR: {best_mrr:.4f}')

    kge_models[model_type] = model
    kge_histories[model_type] = history

# Loss curve comparison
fig, axes = plt.subplots(1, 2, figsize=(12, 4))
for mt, h in kge_histories.items():
    axes[0].plot(h['train_loss'], label=mt.upper())
axes[0].set_xlabel('Epoch')
axes[0].set_ylabel('Loss')
axes[0].set_title('KGE Training Loss')
axes[0].legend()

# Also plot GAT loss
axes[1].plot(gat_history['train_loss'], label='GAT', color='tab:green')
axes[1].set_xlabel('Epoch')
axes[1].set_ylabel('Loss')
axes[1].set_title('GAT Training Loss')
axes[1].legend()
plt.tight_layout()
plt.show()

In [ ]:
# Cell 6: Graph Embedding Extraction

@torch.no_grad()
def extract_embeddings(gat_model, kge_models, loader):
    """Extract per-question graph embeddings from all models.

    Returns dict with:
        'gat': [N, 384] LPG-GAT embeddings
        'transe': [N, 384] RDF-TransE embeddings
        'distmult': [N, 384] RDF-DistMult embeddings
        'questions': list of question strings
        'answers': list of answer strings
        'question_ids': list of question IDs
        'categories': list of category strings
    """
    gat_model.eval()
    for m in kge_models.values():
        m.eval()

    all_gat, all_transe, all_distmult = [], [], []
    all_questions, all_answers, all_qids, all_cats = [], [], [], []

    for batch in loader:
        batch_gpu = batch.to(device)

        # GAT embedding
        gat_emb = gat_model(batch_gpu.lpg_x, batch_gpu.lpg_edge_index, batch_gpu.lpg_batch)
        all_gat.append(gat_emb.cpu())

        # KGE embeddings
        for name, model in kge_models.items():
            emb = model(batch_gpu)
            if name == 'transe':
                all_transe.append(emb.cpu())
            else:
                all_distmult.append(emb.cpu())

        all_questions.extend(batch.questions)
        all_answers.extend(batch.answers)
        all_qids.extend(batch.question_ids)
        all_cats.extend(batch.categories)

    return {
        'gat': F.normalize(torch.cat(all_gat, dim=0), dim=-1),
        'transe': F.normalize(torch.cat(all_transe, dim=0), dim=-1),
        'distmult': F.normalize(torch.cat(all_distmult, dim=0), dim=-1),
        'questions': all_questions,
        'answers': all_answers,
        'question_ids': all_qids,
        'categories': all_cats,
    }


print('Extracting embeddings...')
train_emb = extract_embeddings(gat_model, kge_models, train_loader)
val_emb   = extract_embeddings(gat_model, kge_models, val_loader)
test_emb  = extract_embeddings(gat_model, kge_models, test_loader)

print(f'  Train: {train_emb["gat"].shape[0]} samples')
print(f'  Val:   {val_emb["gat"].shape[0]} samples')
print(f'  Test:  {test_emb["gat"].shape[0]} samples')
print(f'  Embedding dim: {train_emb["gat"].shape[1]}')

In [ ]:
# Cell 7: Question-Answer Embedding Baseline

from sentence_transformers import SentenceTransformer

st_model = SentenceTransformer('all-MiniLM-L6-v2', device=str(device))


def encode_texts(texts, batch_size=64):
    """Encode texts with sentence-transformers → [N, 384] normalized."""
    embs = st_model.encode(texts, batch_size=batch_size, show_progress_bar=False,
                           convert_to_tensor=True, normalize_embeddings=True)
    return embs.cpu()


print('Encoding answers with sentence-transformers...')
test_answer_emb = encode_texts(test_emb['answers'])
test_question_emb = encode_texts(test_emb['questions'])

# Quick check: cosine similarity between graph embeddings and answer embeddings
print('\nMean cosine similarity (graph_emb · answer_emb):')
for name in ['gat', 'transe', 'distmult']:
    cos_sim = (test_emb[name] * test_answer_emb).sum(dim=-1).mean().item()
    print(f'  {name:10s}: {cos_sim:.4f}')

# Baseline: question-only embedding
q_cos = (test_question_emb * test_answer_emb).sum(dim=-1).mean().item()
print(f'  {"question":10s}: {q_cos:.4f} (text-only baseline)')

In [ ]:
# Cell 8: Retrieval Evaluation

def retrieval_eval(query_emb, corpus_emb, ks=(1, 5, 10)):
    """Compute retrieval metrics: Recall@K and MRR.

    Each query[i] should retrieve corpus[i] (diagonal = ground truth).

    Args:
        query_emb: [N, D] query embeddings (graph or question)
        corpus_emb: [N, D] corpus embeddings (answers)
        ks: tuple of K values for Recall@K

    Returns:
        dict with 'mrr' and 'recall@k' for each k
    """
    # Similarity matrix [N, N]
    sim = query_emb @ corpus_emb.T
    N = sim.shape[0]

    # Rank of the correct answer (diagonal)
    diag = sim.diag().unsqueeze(1)  # [N, 1]
    ranks = (sim >= diag).sum(dim=1).float()  # [N] — 1-based rank

    results = {'mrr': (1.0 / ranks).mean().item()}
    for k in ks:
        results[f'recall@{k}'] = (ranks <= k).float().mean().item()
    return results


# Evaluate all models
print(f'{"Model":<12} {"MRR":>8} {"R@1":>8} {"R@5":>8} {"R@10":>8}')
print('-' * 48)

retrieval_results = {}
for name in ['gat', 'transe', 'distmult']:
    r = retrieval_eval(test_emb[name], test_answer_emb)
    retrieval_results[name] = r
    print(f'{name:<12} {r["mrr"]:8.4f} {r["recall@1"]:8.4f} {r["recall@5"]:8.4f} {r["recall@10"]:8.4f}')

# Baseline: question text → answer text retrieval
r_baseline = retrieval_eval(test_question_emb, test_answer_emb)
retrieval_results['question'] = r_baseline
print(f'{"question":<12} {r_baseline["mrr"]:8.4f} {r_baseline["recall@1"]:8.4f} {r_baseline["recall@5"]:8.4f} {r_baseline["recall@10"]:8.4f}')
print('  (question = text-only baseline, no graph)')

# Bar chart
fig, ax = plt.subplots(figsize=(10, 5))
models = list(retrieval_results.keys())
metrics = ['mrr', 'recall@1', 'recall@5', 'recall@10']
x = np.arange(len(models))
width = 0.2

for i, m in enumerate(metrics):
    vals = [retrieval_results[model][m] for model in models]
    ax.bar(x + i * width, vals, width, label=m.upper())

ax.set_xticks(x + width * 1.5)
ax.set_xticklabels([m.upper() for m in models])
ax.set_ylabel('Score')
ax.set_title('Graph → Answer Retrieval Performance')
ax.legend()
ax.set_ylim(0, 1)
plt.tight_layout()
plt.show()

In [ ]:
# Cell 9: Category-wise Analysis

def category_retrieval_eval(emb_dict, answer_emb, categories):
    """Evaluate retrieval per category.

    Returns DataFrame: rows=categories, columns=model×metric.
    """
    cats = sorted(set(categories))
    cat_array = np.array(categories)
    rows = []

    for cat in cats:
        mask = cat_array == cat
        n = mask.sum()
        if n < 2:
            continue
        indices = np.where(mask)[0]

        row = {'category': cat, 'n': int(n)}
        for model_name in ['gat', 'transe', 'distmult']:
            q_emb = emb_dict[model_name][indices]
            a_emb = answer_emb[indices]
            r = retrieval_eval(q_emb, a_emb)
            for metric, val in r.items():
                row[f'{model_name}_{metric}'] = val
        rows.append(row)

    return pd.DataFrame(rows)


cat_df = category_retrieval_eval(test_emb, test_answer_emb, test_emb['categories'])
print('Category-wise MRR:')
print(cat_df[['category', 'n', 'gat_mrr', 'transe_mrr', 'distmult_mrr']].to_string(index=False))

# Bar chart: category × model MRR
fig, ax = plt.subplots(figsize=(14, 5))
cats = cat_df['category'].values
x = np.arange(len(cats))
width = 0.25

for i, model in enumerate(['gat', 'transe', 'distmult']):
    vals = cat_df[f'{model}_mrr'].values
    ax.bar(x + i * width, vals, width, label=model.upper())

ax.set_xticks(x + width)
ax.set_xticklabels(cats, rotation=30, ha='right')
ax.set_ylabel('MRR')
ax.set_title('Category-wise Graph → Answer Retrieval MRR')
ax.legend()
plt.tight_layout()
plt.show()

# Heatmap: model advantage per category
print('\nBest model per category (MRR):')
for _, row in cat_df.iterrows():
    mrrs = {m: row[f'{m}_mrr'] for m in ['gat', 'transe', 'distmult']}
    best = max(mrrs, key=mrrs.get)
    print(f'  {row["category"]:25s} → {best.upper()} ({mrrs[best]:.4f})')

In [ ]:
# Cell 10: Graph Size Effect

def compute_per_sample_rank(query_emb, corpus_emb):
    """Compute rank of correct answer for each sample."""
    sim = query_emb @ corpus_emb.T
    diag = sim.diag().unsqueeze(1)
    ranks = (sim >= diag).sum(dim=1).float()
    return ranks.numpy()


# Collect graph sizes from test set
test_lpg_nodes = []
test_lpg_edges = []
test_rdf_triples = []

for i in range(len(test_ds)):
    d = test_ds[i]
    test_lpg_nodes.append(d.lpg_num_nodes.item())
    test_lpg_edges.append(d.lpg_edge_index.shape[1])
    test_rdf_triples.append(d.rdf_edge_index.shape[1])

test_lpg_nodes = np.array(test_lpg_nodes)
test_lpg_edges = np.array(test_lpg_edges)
test_rdf_triples = np.array(test_rdf_triples)

# Compute per-sample reciprocal rank
rr = {}
for name in ['gat', 'transe', 'distmult']:
    ranks = compute_per_sample_rank(test_emb[name], test_answer_emb)
    rr[name] = 1.0 / ranks

# Scatter plots
fig, axes = plt.subplots(1, 3, figsize=(16, 4.5))

for ax, (name, size_arr, size_label) in zip(axes, [
    ('gat', test_lpg_nodes, 'LPG Nodes'),
    ('transe', test_rdf_triples, 'RDF Triples'),
    ('distmult', test_rdf_triples, 'RDF Triples'),
]):
    ax.scatter(size_arr, rr[name], alpha=0.3, s=10)
    ax.set_xlabel(size_label)
    ax.set_ylabel('Reciprocal Rank')
    ax.set_title(f'{name.upper()}')

    # Trend line via binning
    bins = np.percentile(size_arr, np.linspace(0, 100, 11))
    bins = np.unique(bins)
    if len(bins) >= 2:
        bin_idx = np.digitize(size_arr, bins) - 1
        bin_idx = np.clip(bin_idx, 0, len(bins) - 2)
        bin_centers = []
        bin_means = []
        for b in range(len(bins) - 1):
            mask = bin_idx == b
            if mask.sum() > 0:
                bin_centers.append((bins[b] + bins[b+1]) / 2)
                bin_means.append(rr[name][mask].mean())
        ax.plot(bin_centers, bin_means, 'r-o', linewidth=2, markersize=4, label='Binned mean')
        ax.legend()

plt.suptitle('Graph Size vs Retrieval Quality', y=1.02)
plt.tight_layout()
plt.show()

# Correlation
print('Spearman correlation (graph size vs reciprocal rank):')
from scipy.stats import spearmanr
for name, sizes in [('gat', test_lpg_nodes), ('transe', test_rdf_triples), ('distmult', test_rdf_triples)]:
    corr, pval = spearmanr(sizes, rr[name])
    print(f'  {name:10s}: rho={corr:.3f}, p={pval:.3e}')

In [ ]:
# Cell 11: TransE vs DistMult Deep Dive

# 1. Loss curve comparison
fig, axes = plt.subplots(1, 2, figsize=(12, 4))

axes[0].plot(kge_histories['transe']['train_loss'], label='TransE')
axes[0].plot(kge_histories['distmult']['train_loss'], label='DistMult')
axes[0].set_xlabel('Epoch')
axes[0].set_ylabel('Loss')
axes[0].set_title('Training Loss Comparison')
axes[0].legend()

# Val MRR over time
eval_epochs_kge = [e for e in range(1, KGE_EPOCHS+1) if e % 10 == 0 or e == 1]
for mt in ['transe', 'distmult']:
    n = len(kge_histories[mt]['val_mrr'])
    axes[1].plot(eval_epochs_kge[:n], kge_histories[mt]['val_mrr'][:n], '-o', label=mt.upper())
axes[1].set_xlabel('Epoch')
axes[1].set_ylabel('Val MRR')
axes[1].set_title('Validation MRR Over Training')
axes[1].legend()
plt.tight_layout()
plt.show()


# 2. Per-sample comparison: where does one beat the other?
transe_rr = rr['transe']
distmult_rr = rr['distmult']
diff = transe_rr - distmult_rr  # positive = TransE better

transe_wins = (diff > 0).sum()
distmult_wins = (diff < 0).sum()
ties = (diff == 0).sum()
print(f'Per-sample comparison (test set, N={len(diff)}):')
print(f'  TransE wins:  {transe_wins} ({100*transe_wins/len(diff):.1f}%)')
print(f'  DistMult wins: {distmult_wins} ({100*distmult_wins/len(diff):.1f}%)')
print(f'  Ties:         {ties} ({100*ties/len(diff):.1f}%)')


# 3. Category-level TransE vs DistMult advantage
print('\nCategory-level advantage (MRR difference = TransE - DistMult):')
for _, row in cat_df.iterrows():
    delta = row['transe_mrr'] - row['distmult_mrr']
    arrow = '→ TransE' if delta > 0 else '→ DistMult'
    print(f'  {row["category"]:25s}: {delta:+.4f} {arrow}')


# 4. t-SNE visualization of embedding spaces
from sklearn.manifold import TSNE

fig, axes = plt.subplots(1, 3, figsize=(16, 5))

# Use test set embeddings (subsample if needed)
n_vis = min(300, test_emb['gat'].shape[0])
vis_idx = np.random.choice(test_emb['gat'].shape[0], n_vis, replace=False)
vis_cats = np.array(test_emb['categories'])[vis_idx]
unique_cats = sorted(set(vis_cats))
cat_colors = {c: plt.cm.tab10(i) for i, c in enumerate(unique_cats)}
colors = [cat_colors[c] for c in vis_cats]

for ax, name in zip(axes, ['gat', 'transe', 'distmult']):
    emb = test_emb[name][vis_idx].numpy()
    tsne = TSNE(n_components=2, perplexity=30, random_state=42)
    proj = tsne.fit_transform(emb)

    for cat in unique_cats:
        mask = vis_cats == cat
        ax.scatter(proj[mask, 0], proj[mask, 1], c=[cat_colors[cat]],
                   s=15, alpha=0.6, label=cat[:15])
    ax.set_title(f'{name.upper()} Embedding Space')
    ax.set_xticks([])
    ax.set_yticks([])

axes[2].legend(bbox_to_anchor=(1.05, 1), loc='upper left', fontsize=8)
plt.suptitle('t-SNE of Graph Embeddings (colored by category)', y=1.02)
plt.tight_layout()
plt.show()

In [ ]:
# Cell 12: Summary & Conclusions

print('=' * 60)
print('SUMMARY: G-Retrieval Style Comparison')
print('=' * 60)

# Overall retrieval results table
results_df = pd.DataFrame([
    {'model': name, **metrics}
    for name, metrics in retrieval_results.items()
    if name != 'question'
])
print('\n--- Graph → Answer Retrieval (Test Set) ---')
print(results_df.to_string(index=False, float_format='%.4f'))

# Best model
best_model = results_df.loc[results_df['mrr'].idxmax(), 'model']
print(f'\nBest overall model: {best_model.upper()} (MRR={results_df["mrr"].max():.4f})')

# Category breakdown summary
print('\n--- Best Model per Category (MRR) ---')
category_results = []
for _, row in cat_df.iterrows():
    mrrs = {m: row[f'{m}_mrr'] for m in ['gat', 'transe', 'distmult']}
    best = max(mrrs, key=mrrs.get)
    category_results.append({'category': row['category'], 'best_model': best,
                             'mrr': mrrs[best], 'n': row['n']})
    print(f'  {row["category"]:25s} → {best.upper():10s} (MRR={mrrs[best]:.4f}, n={int(row["n"])})')

category_results = pd.DataFrame(category_results)

# Model strengths
print('\n--- Model Strengths & Weaknesses ---')
print('GAT (LPG):      Pre-computed 384d node features + message passing.')
print('                 Best for: categories with rich LPG structure.')
print('TransE (RDF):    Translation h+r≈t. Asymmetric, handles directed relations.')
print('                 Best for: categories with directional relationships (OWNS, REPORTED).')
print('DistMult (RDF):  Bilinear h·r·t. Symmetric, simpler training dynamics.')
print('                 Best for: symmetric or co-occurrence patterns.')

# Verification assertions
assert len(results_df) == 3, f'Expected 3 models, got {len(results_df)}'
assert all(col in results_df.columns for col in ['model', 'mrr', 'recall@1', 'recall@5'])
assert len(category_results) >= 1, 'No category results'
print(f'\nVerification passed: {len(results_df)} models, {len(category_results)} categories.')